In [1]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer
from sklearn.metrics import f1_score
import numpy as np

In [3]:
# 定義模型和tokenizer
model_name_or_path = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
model_BERT = BertForQuestionAnswering.from_pretrained(model_name_or_path)
tokenizer_BERT = BertTokenizer.from_pretrained(model_name_or_path)

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

c:\Users\PARALELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PARALELL\.cache\huggingface\hub\models--google-bert--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
model_name_or_path = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/distilbert-base-uncased-distilled-squad"
model_distilBert = DistilBertForQuestionAnswering.from_pretrained(model_name_or_path)
tokenizer_distilBert = DistilBertTokenizer.from_pretrained(model_name_or_path)

In [5]:
from datasets import load_dataset

# 加載IMDb資料集
dataset = load_dataset('imdb')

In [7]:
questions = [
    "What is the overall sentiment of the review?",
    "How does the reviewer descibe the main actor`s performances?",
    "What does the reviewer say about the plot design?",
]

context = dataset["train"][0]["text"]
print(context)

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [12]:
def get_answer(model, tokenizer, question, context):
    inputs = tokenizer.encode_plus(question,context,return_tensors = "pt") # ??
    with torch.no_grad():
        outputs = model(**inputs)
    ansewer_start = torch.argmax(outputs.start_logits)
    ansewer_end = torch.argmax(outputs.end_logits)
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][ansewer_start:ansewer_end]))
    return answer

In [13]:
for question in questions:
    bert_answer = get_answer(model_BERT, tokenizer_BERT, question, context)
    print(f"BERT回答 : {bert_answer}")

BERT回答 : 
BERT回答 : 
BERT回答 : doesn ' t have much of a


In [ ]:
# ==============================================================================================================================
# 需要根據真實影評，反向設計題目
questions = [
    #"What does the reviewer commend the filmmakers for?",
    "What is the overall sentiment of the review?",
    "How does the reviewer descibe the main actor`s performances?",
    "What does the reviewer say about the plot design?",
]

import re
# 清理文本函數
def clean_text(text):
    # 去除HTML標籤
    text = re.sub(r'<[^>]+>', '', text)
    return text

context = clean_text(dataset["train"][0]["text"])
print("context :\n",context)

# ==============================================================================================================================
model = model_BERT
tokenizer = tokenizer_BERT
question = questions[0]

# ==============================================================================================================================
inputs = tokenizer.encode_plus(question,context,return_tensors = "pt") # ??
#print("inputs :\n",inputs)
with torch.no_grad():
    outputs = model(**inputs)
ansewer_start = torch.argmax(outputs.start_logits)
ansewer_end = torch.argmax(outputs.end_logits) + 1 
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][ansewer_start:ansewer_end]))
print("answer :",answer) # 若模型無法找到合適的答案，則會返回 [CLS]

context :
 I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [ ]:
# 評估指標示範（假設真實答案已知）
true_answers = [
    "positive",
    "excellent",
    "well-designed",
]

def coupute_em_f1 (pred, true):
    prd = pred.lower().strip()
    ture = true.lower().strip()
    em = int(pred == true)

    pred_tokens = pred.split()
    true_tokens = true.split()
    common = set(pred_tokens) & set(true_tokens)
    f1 = (2 * len(common)) / len(pred_tokens) + len(true_tokens)
    if (len(pred_tokens) + len(true_tokens) > 0) :
        return em, f1
    else:
        0

for i, question in enumerate(questions):
    BERT_em, BERT_f1 = coupute_em_f1(get_answer(model_BERT, tokenizer_BERT, question, context), true_answers[i])
    distilBert_em, distilBert_f1 = coupute_em_f1(get_answer(model_distilBert, tokenizer_distilBert, question, context), true_answers[i])
    print(f"問題 :{question}")
    print(f"BERT-EM : {BERT_em}, F1 :{BERT_f1}")
    print(f"DistilBert-EM : {distilBert_em}, F1 :{distilBert_f1}")
